In [1]:
# Set up the environment
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# The goal
What are common features among colleges and universities that have closed in the last decade?
Using these features, is it possible to predict which schools are in danger of closing in the near future?
Are there steps that schools can take to mitigate the features which may lead to their closing?

# Data collection
The data come from the US Department of Education College Scorecard, which is available on Kaggle and from the US Department of Education at https://collegescorecard.ed.gov/data/.

In [2]:
# Load the most recent scorecard into a DataFrame for examination
# List of fields I want to import, based on previous exploration and data dictionary
fields = ['Id','OPEID','INSTNM','CITY', 'ZIP','sch_deg', 'main', 'NUMBRANCH', 'PREDDEG', \
         'HIGHDEG', 'CONTROL','st_fips','region','LOCALE','CCUGPROF','RELAFFIL', 'ADM_RATE',\
         'ACTCMMID','SAT_AVG', 'UGDS', 'UG', 'CURROPER', 'NPT4_PUB', 'NPT4_PRIV', 'COSTT4_A', \
         'COSTT4_P','TUITFTE', 'INEXPFTE', 'AVGFACSAL', 'PFTFAC',\
         'C150_4', 'C150_L4','RET_FT4','RET_FTL4','RET_PT4','RET_PTL4','PCTFLOAN','UG25abv',\
         'COMP_ORIG_YR4_RT','WDRAW_ORIG_YR4_RT', 'ENRL_ORIG_YR4_RT', 'INC_PCT_LO', 'DEP_STAT_PCT_IND',\
         'IND_INC_PCT_LO', 'DEP_INC_PCT_LO', 'PAR_ED_PCT_1STGEN', 'DEP_INC_AVG', 'IND_INC_AVG', \
         'DEBT_MDN', 'GRAD_DEBT_MDN', 'WDRAW_DEBT_MDN', 'loan_ever', 'age_entry', 'female',\
         'married', 'dependent', 'veteran', 'faminc', 'md_faminc', 'median_hh_inc']

data = pd.read_csv("data/raw/Scorecard.csv", usecols=fields, low_memory=False)
data.shape

(124699, 60)

## Data organization
Create a file structure (and add work to GitHub repo)

In [3]:
#Create directories for various components of the project
#os.mkdir("data")
#os.mkdir("data/raw")
#os.mkdir("data/interim")
#os.mkdir("data/processed")
#os.mkdir("data/external")
#os.mkdir("figures")
#os.mkdir("models")

In [4]:
# Look at basic info about the dataframe - column names, summary information
data.columns

Index(['Id', 'OPEID', 'INSTNM', 'CITY', 'ZIP', 'sch_deg', 'main', 'NUMBRANCH',
       'PREDDEG', 'HIGHDEG', 'CONTROL', 'st_fips', 'region', 'LOCALE',
       'CCUGPROF', 'RELAFFIL', 'ADM_RATE', 'ACTCMMID', 'SAT_AVG', 'UGDS', 'UG',
       'CURROPER', 'NPT4_PUB', 'NPT4_PRIV', 'COSTT4_A', 'COSTT4_P', 'TUITFTE',
       'INEXPFTE', 'AVGFACSAL', 'PFTFAC', 'C150_4', 'C150_L4', 'RET_FT4',
       'RET_FTL4', 'RET_PT4', 'RET_PTL4', 'PCTFLOAN', 'UG25abv',
       'COMP_ORIG_YR4_RT', 'WDRAW_ORIG_YR4_RT', 'ENRL_ORIG_YR4_RT',
       'INC_PCT_LO', 'DEP_STAT_PCT_IND', 'DEP_INC_PCT_LO', 'IND_INC_PCT_LO',
       'PAR_ED_PCT_1STGEN', 'DEP_INC_AVG', 'IND_INC_AVG', 'DEBT_MDN',
       'GRAD_DEBT_MDN', 'WDRAW_DEBT_MDN', 'loan_ever', 'age_entry', 'female',
       'married', 'dependent', 'veteran', 'faminc', 'md_faminc',
       'median_hh_inc'],
      dtype='object')

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124699 entries, 0 to 124698
Data columns (total 60 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Id                 124699 non-null  int64  
 1   OPEID              124699 non-null  object 
 2   INSTNM             124699 non-null  object 
 3   CITY               124699 non-null  object 
 4   ZIP                124699 non-null  object 
 5   sch_deg            101334 non-null  float64
 6   main               124699 non-null  object 
 7   NUMBRANCH          124699 non-null  int64  
 8   PREDDEG            124699 non-null  object 
 9   HIGHDEG            124699 non-null  object 
 10  CONTROL            124676 non-null  object 
 11  st_fips            124669 non-null  object 
 12  region             124669 non-null  object 
 13  LOCALE             7380 non-null    object 
 14  CCUGPROF           3559 non-null    object 
 15  RELAFFIL           917 non-null     object 
 16  AD

In [6]:
data.nunique()

Id                   124699
OPEID                 11511
INSTNM                20601
CITY                   5494
ZIP                   16527
sch_deg                   3
main                      2
NUMBRANCH                58
PREDDEG                   5
HIGHDEG                   5
CONTROL                   3
st_fips                  59
region                   10
LOCALE                   12
CCUGPROF                 14
RELAFFIL                 59
ADM_RATE               7717
ACTCMMID                 32
SAT_AVG                 784
UGDS                  13472
UG                     2804
CURROPER                  2
NPT4_PUB               6887
NPT4_PRIV             13237
COSTT4_A              15353
COSTT4_P               9594
TUITFTE               23870
INEXPFTE              19612
AVGFACSAL              9870
PFTFAC                 8688
C150_4                 7985
C150_L4                8989
RET_FT4                5065
RET_FTL4               5289
RET_PT4                1841
RET_PTL4            

Many of the columns that I expected to be numerical data were actually classifed as strings. I tried to convert them to numbers, but got an error message, so I explored the content of the colummns and the data dictionary to see what the problem was.

For privacy reasons, some of the columns include data is coded as "PrivacySupressed".

In [7]:
# Look at individual column information to assess and assign data types
print((data.COMP_ORIG_YR4_RT.value_counts()/len(data.COMP_ORIG_YR4_RT)) *100)
print((data.COMP_ORIG_YR4_RT.isnull().sum() /len(data.COMP_ORIG_YR4_RT)) *100)

PrivacySuppressed    11.696966
0                     3.828419
1                     0.334405
0.5                   0.288695
0.666666666667        0.158782
                       ...    
0.282692307692        0.000802
0.283106796117        0.000802
0.39514978602         0.000802
0.281588447653        0.000802
0.545112781955        0.000802
Name: COMP_ORIG_YR4_RT, Length: 32741, dtype: float64
31.17186184331871


In [8]:
# Replace "PrivacySuppressed" with null values (NaN)
for col in data.columns:
    if data[col].dtype == 'object':
        data[col] = data[col].str.replace("PrivacySuppressed", "NaN")

In [9]:
# Look at percentage of null values per column, now that PrivacySupressed has been recoded
nas=pd.DataFrame(data.isnull().sum().sort_values(ascending=False)/len(data),columns = ['percent'])
pos = nas['percent'] > 0
nas[pos]

,percent
RELAFFIL,0.992646
CCUGPROF,0.971459
UG,0.950737
LOCALE,0.940817
NPT4_PUB,0.926575
COSTT4_P,0.901715
RET_PT4,0.876687
ACTCMMID,0.872726
NPT4_PRIV,0.851162
SAT_AVG,0.850857


In [10]:
# Get a list of columns with > 90% NAN values
to_drop = [column for column in data.columns if (data[column].isnull().sum()/len(data[column]) > 0.90)]
#data.isnull().sum()/len(data)

print("Columns to drop include: ", to_drop)

Columns to drop include:  ['LOCALE', 'CCUGPROF', 'RELAFFIL', 'UG', 'NPT4_PUB', 'COSTT4_P']


In [11]:
# Remove columns with > 90% NAN values from dataframe
data = data.drop(to_drop, axis=1)
data.head()

,Id,OPEID,INSTNM,CITY,ZIP,sch_deg,main,NUMBRANCH,PREDDEG,HIGHDEG,...,WDRAW_DEBT_MDN,loan_ever,age_entry,female,married,dependent,veteran,faminc,md_faminc,median_hh_inc
0,1,01230800,COMMUNITY COLLEGE OF THE AIR FORCE,MONTGOMERY,361126613,NaN,Main campus,1,Predominantly associate's-degree granting,Associate degree,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,00100200,ALABAMA A & M UNIVERSITY,NORMAL,35762,NaN,Main campus,1,Predominantly bachelor's-degree granting,Graduate degree,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,00105200,UNIVERSITY OF ALABAMA AT BIRMINGHAM,BIRMINGHAM,352940110,NaN,Main campus,2,Predominantly bachelor's-degree granting,Graduate degree,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,00574900,ALABAMA AVIATION AND TECHNICAL COLLEGE,OZARK,36360,NaN,Main campus,1,Predominantly certificate-degree granting,Associate degree,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,02503400,SOUTHERN CHRISTIAN UNIVERSITY,MONTGOMERY,361173553,NaN,Main campus,1,Predominantly bachelor's-degree granting,Graduate degree,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# Convert column types, based on data dictionary
#data.CCUGPROF = data.CCUGPROF.astype('category') # dropped
#data.RELAFFIL = data.RELAFFIL.astype('category') #dropped
#data.LOCALE = data.LOCALE.astype('category')
data.PREDDEG = data.PREDDEG.astype('category')
data.HIGHDEG = data.HIGHDEG.astype('category')
data.CONTROL = data.CONTROL.astype('category')
data.st_fips = data.st_fips.astype('category')
data.region = data.region.astype('category')
data.main = data.main.astype('bool')
data.COMP_ORIG_YR4_RT = data.COMP_ORIG_YR4_RT.astype('float')
data.WDRAW_ORIG_YR4_RT = data.WDRAW_ORIG_YR4_RT.astype('float')
data.ENRL_ORIG_YR4_RT= data.ENRL_ORIG_YR4_RT.astype('float')
data.INC_PCT_LO = data.INC_PCT_LO.astype('float')
data.DEP_STAT_PCT_IND= data.DEP_STAT_PCT_IND.astype('float')
data.DEP_INC_PCT_LO = data.DEP_INC_PCT_LO.astype('float')
data.IND_INC_PCT_LO = data.IND_INC_PCT_LO.astype('float')
data.PAR_ED_PCT_1STGEN = data.PAR_ED_PCT_1STGEN.astype('float')
data.DEP_INC_AVG = data.DEP_INC_AVG.astype('float')
data.IND_INC_AVG = data.IND_INC_AVG.astype('float')
data.DEBT_MDN = data.DEBT_MDN.astype('float')
data.GRAD_DEBT_MDN = data.GRAD_DEBT_MDN.astype('float')
data.WDRAW_DEBT_MDN = data.WDRAW_DEBT_MDN.astype('float')
data.loan_ever = data.loan_ever.astype('float')
data.age_entry = data.age_entry.astype('float')
data.female = data.female.astype('float')
data.married = data.married.astype('float')
data.dependent = data.dependent.astype('float')
data.veteran = data.veteran.astype('float')
data.faminc = data.faminc.astype('float')
data.md_faminc = data.md_faminc.astype('float')
data.median_hh_inc = data.median_hh_inc.astype('float')

In [13]:
#Rename columns to be more descriptive
col_names = ['Id','OPEID','Name','City','Zip','Predominant degree', 'Main branch', 'NumBranches',\
            'Primary degree type', 'High degree', 'Control', 'StateCode','Region', \
            'AdmissionRate', 'ACTMidpoint', 'SATAvg',\
            'Enrollment-DegreeSeeking', \
            'CURROPER','NetPrice-Private', 'AvgCost-AY', 'TuitionRevenue/FTE',\
            'Expenditure/FTE','AvgFaculty Salary', 'Percent Fulltime Faculty', 'Completion-4yr', 'Completion<4yr','RetentionFT-4yr', \
            'RetentionFT-<4yr','RetentionPT-4yr', 'RetentionPT-<4yr', 'PercentFedLoan','PercentAge>25', \
            'PercentCompleted4yr', 'PercentWithdraw4yr', 'PercentEnrolled4yr',\
            'PercentAidedLowIncome', 'PercentIndependent', 'PercentDependentLowIncome','PercentIndepentdentLowIncome',\
            'PercentFirstGen','AvgIncomeDependent','AvgIncomeIndep','MedianDebt', \
            'MedianDebtCompleters', 'MedianDebtNotComplete', 'PercentReceiveFedLoan','AvgAgeEntry', 'PercentFemale',\
            'PercentMarried','PercentDependent','PercentVeteran', 'AvgFamilyIncome','MedianFamilyIncome',\
            'MedianHouseholdIncome']
data.columns = col_names
data.set_index('Id').head()

,OPEID,Name,City,Zip,Predominant degree,Main branch,NumBranches,Primary degree type,High degree,Control,...,MedianDebtNotComplete,PercentReceiveFedLoan,AvgAgeEntry,PercentFemale,PercentMarried,PercentDependent,PercentVeteran,AvgFamilyIncome,MedianFamilyIncome,MedianHouseholdIncome
Id,,,,,,,,,,,,,,,,,,,,,
1,01230800,COMMUNITY COLLEGE OF THE AIR FORCE,MONTGOMERY,361126613,NaN,True,1,Predominantly associate's-degree granting,Associate degree,Public,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,00100200,ALABAMA A & M UNIVERSITY,NORMAL,35762,NaN,True,1,Predominantly bachelor's-degree granting,Graduate degree,Public,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00105200,UNIVERSITY OF ALABAMA AT BIRMINGHAM,BIRMINGHAM,352940110,NaN,True,2,Predominantly bachelor's-degree granting,Graduate degree,Public,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00574900,ALABAMA AVIATION AND TECHNICAL COLLEGE,OZARK,36360,NaN,True,1,Predominantly certificate-degree granting,Associate degree,Public,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,02503400,SOUTHERN CHRISTIAN UNIVERSITY,MONTGOMERY,361173553,NaN,True,1,Predominantly bachelor's-degree granting,Graduate degree,Private nonprofit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# Look for duplicate rows
duplicateRowsDF = data[data.duplicated()]
duplicateRowsDF

,Id,OPEID,Name,City,Zip,Predominant degree,Main branch,NumBranches,Primary degree type,High degree,...,MedianDebtNotComplete,PercentReceiveFedLoan,AvgAgeEntry,PercentFemale,PercentMarried,PercentDependent,PercentVeteran,AvgFamilyIncome,MedianFamilyIncome,MedianHouseholdIncome


In [15]:
# Write complete dataframe to file.
data.to_csv('data/interim/college.filtered.csv')

In [16]:
# Convert CURROPER from category to Boolean
curroper = {"Currently certified as operating": 1, "Not currently certified as an operating institution": 0}
data['CURROPER'] = data['CURROPER'].replace(curroper)
data['CURROPER'] = data['CURROPER'].astype('bool')
print(data['CURROPER'])

# Write complete dataframe to file.
data.to_csv('data/processed/college.filtered.csv')
# Create a dataframe with only closed schools
closed = data[~data['CURROPER']]
operational = data[data['CURROPER']]
print("Data: ", data.shape)
print("Open schools: ", operational.shape)
print("Closed schools: ", closed.shape)

0          True
1          True
2          True
3         False
4          True
          ...  
124694     True
124695     True
124696     True
124697     True
124698     True
Name: CURROPER, Length: 124699, dtype: bool
Data:  (124699, 54)
Open schools:  (101121, 54)
Closed schools:  (23578, 54)


In [17]:
print(len(closed.OPEID.unique()))

3826


In [18]:
closed.to_csv('data/interim/closed.filtered.csv')
operational.to_csv('data/interim/operational.filtered.csv')

In [19]:
# Counts and percents unique values
# Ranges of values

closed.describe().T

,count,mean,std,min,25%,50%,75%,max
Id,23578.0,49811.772330,33055.783377,4.000000,20497.250000,45606.500000,75408.250000,1.246920e+05
Predominant degree,18503.0,1.789007,0.826354,1.000000,1.000000,2.000000,3.000000,3.000000e+00
NumBranches,23578.0,4.849563,12.537724,1.000000,1.000000,1.000000,3.000000,1.410000e+02
AdmissionRate,4080.0,0.693199,0.286237,0.000000,0.498050,0.700000,0.929050,1.000000e+01
ACTMidpoint,322.0,20.773292,2.951556,9.000000,19.000000,21.000000,23.000000,3.200000e+01
SATAvg,464.0,1001.066810,113.435312,560.000000,937.000000,990.000000,1056.000000,1.405000e+03
Enrollment-DegreeSeeking,20555.0,499.965604,2667.078410,0.000000,55.000000,155.000000,446.500000,2.418320e+05
NetPrice-Private,1906.0,17725.508919,7009.961774,-5029.000000,13507.000000,18237.500000,22505.250000,4.484200e+04
AvgCost-AY,1323.0,23399.270597,7171.311205,3368.000000,19692.000000,24593.000000,27280.000000,4.836100e+04
TuitionRevenue/FTE,17997.0,21438.137912,262530.265546,0.000000,2548.000000,6158.000000,10495.000000,2.667016e+07


In [20]:
closed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23578 entries, 3 to 124691
Data columns (total 54 columns):
 #   Column                        Non-Null Count  Dtype   
---  ------                        --------------  -----   
 0   Id                            23578 non-null  int64   
 1   OPEID                         23578 non-null  object  
 2   Name                          23578 non-null  object  
 3   City                          23578 non-null  object  
 4   Zip                           23578 non-null  object  
 5   Predominant degree            18503 non-null  float64 
 6   Main branch                   23578 non-null  bool    
 7   NumBranches                   23578 non-null  int64   
 8   Primary degree type           23578 non-null  category
 9   High degree                   23578 non-null  category
 10  Control                       23577 non-null  category
 11  StateCode                     23577 non-null  category
 12  Region                        23577 non-null 

In [21]:
# Find the percent of null values in each column
nas=pd.DataFrame(closed.isnull().sum().sort_values(ascending=False)/len(closed),columns = ['percent'])
pos = nas['percent'] > 0
nas[pos]

,percent
ACTMidpoint,0.986343
SATAvg,0.980321
RetentionPT-4yr,0.956697
AvgCost-AY,0.943888
RetentionFT-4yr,0.925693
NetPrice-Private,0.919162
Completion-4yr,0.896980
RetentionPT-<4yr,0.871236
PercentFedLoan,0.853974
AdmissionRate,0.826957


In [22]:
operational.describe().T

,count,mean,std,min,25%,50%,75%,max
Id,101121.0,65273.490986,3.602785e+04,1.000000,34585.000000,66740.000000,96875.000000,1.246990e+05
Predominant degree,82831.0,2.012423,8.819505e-01,1.000000,1.000000,2.000000,3.000000,3.000000e+00
NumBranches,101121.0,3.478387,1.262927e+01,1.000000,1.000000,1.000000,1.000000,1.410000e+02
AdmissionRate,30076.0,0.697474,2.140628e-01,0.000000,0.564375,0.722300,0.855200,3.687700e+00
ACTMidpoint,15549.0,22.763329,3.259005e+00,2.000000,21.000000,22.000000,24.000000,3.500000e+01
SATAvg,18134.0,1054.022223,1.269459e+02,514.000000,971.000000,1036.000000,1117.000000,1.599000e+03
Enrollment-DegreeSeeking,89788.0,2529.405333,5.015666e+03,0.000000,146.000000,705.000000,2556.000000,2.535940e+05
NetPrice-Private,16654.0,18011.659481,7.511786e+03,-103168.000000,13080.500000,18162.000000,22483.500000,8.757000e+04
AvgCost-AY,19181.0,22974.776654,1.181876e+04,2200.000000,13062.000000,20940.000000,29762.000000,7.447300e+04
TuitionRevenue/FTE,88058.0,11785.763179,9.475136e+04,0.000000,2479.000000,5682.000000,10655.000000,1.193246e+07


In [23]:
operational.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 101121 entries, 0 to 124698
Data columns (total 54 columns):
 #   Column                        Non-Null Count   Dtype   
---  ------                        --------------   -----   
 0   Id                            101121 non-null  int64   
 1   OPEID                         101121 non-null  object  
 2   Name                          101121 non-null  object  
 3   City                          101121 non-null  object  
 4   Zip                           101121 non-null  object  
 5   Predominant degree            82831 non-null   float64 
 6   Main branch                   101121 non-null  bool    
 7   NumBranches                   101121 non-null  int64   
 8   Primary degree type           101121 non-null  category
 9   High degree                   101121 non-null  category
 10  Control                       101099 non-null  category
 11  StateCode                     101092 non-null  category
 12  Region                        

In [24]:
nas=pd.DataFrame(operational.isnull().sum().sort_values(ascending=False)/len(operational),columns = ['percent'])
pos = nas['percent'] > 0
nas[pos]

,percent
RetentionPT-4yr,0.858031
ACTMidpoint,0.846234
NetPrice-Private,0.835306
SATAvg,0.820670
AvgCost-AY,0.810316
RetentionPT-<4yr,0.807646
RetentionFT-4yr,0.790874
MedianHouseholdIncome,0.760792
PercentReceiveFedLoan,0.759842
AvgFamilyIncome,0.759842
